In [1]:
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.0 MB/s eta 0:00:00


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.8 MB/s eta 0:00:00


In [3]:
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.3 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [5]:
model_name = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [7]:
# from datasets import load_dataset
import pandas as pd
import json

dataset = pd.read_csv("data_full_v1.csv")
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)
dataset = dataset.sample(frac=1)
dataset = json.loads(dataset.to_json(orient ='records'))

In [8]:
# Format the COPA query for the GPT-3.5 Turbo model
def format_query(item):
    # return f"{item['prompt']}. Yes or No? Don't give any explanation"
    return f"Answer only 1 word either \"Yes\" or \"No\" of the following question - {item['prompt']}."

  # return f"{item['prompt']}. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit"

format_query(dataset[2])

'Answer only 1 word either "Yes" or "No" of the following question - Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Jyka has a direct effect on qwiu and yupt. Yupt has a direct effect on kwox. Qwiu has a direct effect on kwox.\nFor those who are not jyka, the probability of kwox is 25%. For those who are jyka, the probability of kwox is 24%.\nDoes jyka positively affect kwox through yupt and qwiu?.'

In [9]:
# len(tokenizer.tokenize(format_query(dataset[2])))

In [10]:
# import re
# print(len(format_query(dataset[15]))+100)
# response = pipe(format_query(dataset[2]), max_length=200)
# print(response)
# generated_text = response[0]['generated_text']
# # print(generated_text)

# generated_text = generated_text.removeprefix(format_query(dataset[2])).lower()

# print(generated_text)
# generated_text = [w for w in re.split('\W', generated_text) if w]
# # print(generated_text)

In [11]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

# prompt = format_query(dataset[3])
# print(prompt)

# model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
# model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
# tokenizer.batch_decode(generated_ids)[0]

In [12]:
# prompt = format_query(dataset[7])
# print(prompt)

# model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
# model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True)
# tokenizer.batch_decode(generated_ids)[0]

# generated_text = response[0]['generated_text']




# #       print(generated_text)
# generated_text = generated_text.removeprefix(query).lower()

#       # print(generated_text)
# generated_text = [w for w in re.split('\W', generated_text) if w]
# print(generated_text)

In [20]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, RetryError
import re

correct_count = 0
processed_items = 0
processed = []
failed_items = []
y_true = []
y_pred = []

# Evaluate the accuracy of GPT-3.5 Turbo on the COPA dataset
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(1))
def evaluate_accuracy(item):
      global correct_count
      query = format_query(item)

      # model_inputs = tokenizer([query], return_tensors="pt").to(device)
      # model.to(device)
      # generated_ids = model.generate(**model_inputs, max_new_tokens=len(format_query(dataset[15]))+100, do_sample=True)
      # generated_text = tokenizer.batch_decode(generated_ids)[0]
      # print(generated_text)



      # print(len(tokenizer.tokenize(query)))
      response = pipe(query, max_length=len(tokenizer.tokenize(query))+100)
      # print('response', response)
      generated_text = response[0]['generated_text']




#       print(generated_text)
      generated_text = generated_text.removeprefix(query).lower()

      # print(generated_text)
      generated_text = [w for w in re.split('\W', generated_text) if w]
#       print(generated_text)
      # generated_text = ''.join(filter(str.isalnum, generated_text)).split()

      if (('yes' in generated_text) or ('yes.' in generated_text)) and ((not 'no' in generated_text) and (not 'no.' in generated_text)):
        response = 1
      elif (('no' in generated_text) or ('no.' in generated_text)) and ((not 'yes' in generated_text) and (not 'yes.' in generated_text)):
        response = 0
      else:
#         print(query)
        print(response)
        print(generated_text)
        raise Exception("failure")

      # response = pred[-1]
      print(response)
      # print(item['ground_truth'])
      correct_hypothesis = ((int(response) == item['ground_truth'])==True)
      y_true.append(item['ground_truth'])
      y_pred.append(int(response))
      # print(correct_hypothesis)
      correct_count += correct_hypothesis
      print("correct_count is ", correct_count)
      # print("completed\n")


# Evaluate accuracy
def evaluate_accuracy_wrapper(data):
      global correct_count
      global processed_items
      global failed_items
      for item in data:
          try:
              evaluate_accuracy(item)
              processed_items = processed_items +1
              processed.append(item)
          except RetryError as e:
              print("failed")
              failed_items.append(item)
          if processed_items==1000:
            break
          if processed_items%10==0:
            print('processed_items ', processed_items)
      print("correct_count in wrapper is ", correct_count)
      return correct_count / len(data)

accuracy = evaluate_accuracy_wrapper(dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with open("y_true.txt", "w") as f:
    for s in y_true:
        f.write(str(s) +"\n")

In [ ]:
with open("y_pred.txt", "w") as f:
    for s in y_pred:
        f.write(str(s) +"\n")

In [ ]:
with open("processed.txt", "w") as f:
    for s in processed:
        f.write(str(s) +"\n")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.30      0.37       483
           1       0.52      0.70      0.59       517

    accuracy                           0.51      1000
   macro avg       0.50      0.50      0.48      1000
weighted avg       0.50      0.51      0.48      1000

